In [1]:
import pandas as pd
from pathlib import Path
import util

pd.set_option('display.float_format', '{:,.0f}'.format)

In [2]:
# network data
df_network = util.process_network_summary()

In [3]:
county_order = ['King','Kitsap','Pierce','Snohomish','Outside Region','Total']

df_network['county'] = pd.Categorical(df_network['county'], ordered=True,
                   categories=county_order)

In [4]:
#| label: system_summary

# network summary: 'VMT','VHT','total_delay'
vmt = df_network['VMT'].sum()
vht = df_network['VHT'].sum()
delay = df_network['total_delay'].sum()

# Get light rail boardings
df_boardings = pd.read_csv(util.output_path / 'transit/daily_boardings_by_agency.csv')
# transit boardings
transit_ridership = df_boardings['boardings'].sum()
#Light rail boardings
df_line_boardings = pd.read_csv(util.output_path / 'transit/transit_line_results.csv')
lr_ridership = df_line_boardings[df_line_boardings['mode']=='r']['boardings'].sum()

# mode share
df_trip = pd.read_csv(util.output_path / 'agg/dash/mode_share_county.csv')
df = df_trip['trexpfac'].sum() # total trips
transit_share = df_trip.loc[df_trip['mode']=="Transit"]['trexpfac'].sum()/df

# emission
df_emissions = pd.read_csv(util.output_path / 'emissions/emissions_summary.csv')
df = df_emissions.loc[(df_emissions['veh_type'].isin(['light','medium','heavy'])) & \
                      (df_emissions['pollutant_name']=="CO2 Equivalent")].copy()
CO2e = df['total_daily_tons'].sum()

run_summary2 = pd.DataFrame({
    'VMT': [vmt],
    'VHT': [vht],
    'Delay': [delay],
    'Transit Boardings': [transit_ridership],
    'Light Rail Boardings': [lr_ridership],
    '% Transit': [transit_share],
    'CO2e': [CO2e]
})

run_summary2.style.\
        format('{:,.0f}', subset=['VMT','VHT','Delay','Transit Boardings','Light Rail Boardings','CO2e']).\
        format('{:.1%}', subset=['% Transit']).\
            hide(axis="index")

VMT,VHT,Delay,Transit Boardings,Light Rail Boardings,% Transit,CO2e
"98,739,934","3,123,973","333,309","1,546,156","471,632",4.7%,"6,540"


In [5]:
county_order = ['King','Kitsap','Pierce','Snohomish','Outside Region','Total']

df_network['county'] = pd.Categorical(df_network['county'], ordered=True,
                   categories=county_order)

In [6]:
#| label: total_vmt_vht_delay_by_county

df_vmt = df_network.reset_index().groupby('county',observed=True)['VMT'].sum()
df_vht = df_network.reset_index().groupby('county',observed=True)['VHT'].sum()
df_delay = df_network.reset_index().groupby('county',observed=True)['total_delay'].sum()

df = pd.concat([df_vmt,df_vht,df_delay], axis=1)
df = df[df.index!='Outside Region']
df.rename(columns={'total_delay': 'Total Delay Hours'}, inplace=True)

df.loc['Total',] = df.sum()
total_vmt = df.loc['Total','VMT']

df.style.format('{:,.0f}')

,VMT,VHT,Total Delay Hours
county,,,
King,"50,249,616","1,630,468","209,875"
Kitsap,"5,365,382","161,110","6,633"
Pierce,"22,405,356","699,704","55,363"
Snohomish,"20,288,358","623,847","61,410"
Total,"98,308,712","3,115,129","333,281"


In [7]:
#| label: transit
transit = pd.read_csv(util.output_path / 'transit/daily_boardings_by_agency.csv')

df = transit[['agency_name','boardings']].set_index('agency_name').copy()
df.loc['Total',:] = df['boardings'].sum()
df.rename(columns={'boardings': 'Daily Boardings'}, inplace=True)


df.style.format('{:,.0f}')


,Daily Boardings
agency_name,
King County Metro,"778,445"
Sound Transit,"512,202"
Community Transit,"99,474"
Pierce Transit,"71,625"
Kitsap Transit,"43,100"
Washington Ferries,"23,421"
Everett Transit,"17,889"
Total,"1,546,156"


In [8]:
#| label: mode_share
mode_share = pd.read_csv(util.output_path / 'agg/dash/mode_share_county.csv')

df = mode_share.groupby('mode')['trexpfac'].sum().reset_index().set_index('mode')
df['Mode Share'] = (df['trexpfac']/ (mode_share['trexpfac'].sum()))
df.loc['Total', :] = df.sum(axis=0)
df['Mode Share'] = (df['Mode Share'] * 100).round(1)
df['Mode Share'] = df['Mode Share'].astype(str) + '%'
df['trexpfac'] = df['trexpfac'].apply(lambda x: f"{int(round(x)):,}")
df.rename(columns={'trexpfac': 'Total Person Trips'}, inplace=True)
df

,Total Person Trips,Mode Share
mode,,
Bike,"389,931",1.8%
HOV2,"4,357,089",19.8%
HOV3+,"2,748,191",12.5%
SOV,"8,653,544",39.2%
School Bus,"303,430",1.4%
TNC,"126,251",0.6%
Transit,"1,029,688",4.7%
Walk,"4,448,787",20.2%
Total,"22,056,911",100.0%
